In [ ]:
!python -m pip install git+https://github.com/pytube/pytube
!pip uninstall aligo -y

In [ ]:
from pytube import YouTube, Playlist
from threading import Thread, Semaphore
from time import ctime, sleep
from math import log, ceil
from tqdm.notebook import tqdm
import os
import random
import re


In [ ]:
!rm -rf ./*

In [ ]:
class videoDownloader(Thread):
    def __init__(self,video=None,id=''):
        super().__init__()
        self.yt = video
        self.complete = False
        self.title = re.sub('[/:*?"<>|]','|', self.yt.title)
        if len(self.title) > 66:
          self.title = self.title[:66]
        self.id = id
        if id:
          self.id += '_'
    def run(self):
        # semaLock.acquire()
        try:
          if self.yt != None:
              print(ctime() + " " + self.id + self.yt.title + ": downloading ...")
              if  dlvideo:
                  self.downloadVideo()
              elif dlaudio:
                  self.downloadAudio()
              if dlsubtitle:
                  self.downloadSubtitle()
              print(ctime() + " " + self.id + self.yt.title + ": complete.")
              sleep(7)
        except Exception as e:
          print(ctime() + " " + self.id + self.yt.title + ": " + e)
        finally:
          self.complete = True
    def setvideo(self, video):
        self.yt = video
    def downloadVideo(self):
        videoss = self.yt.streams

        if res < 8 and res > -1:
            availableStream = []
            resolution_index = res
            mime_t = limit['mime_type']
            type = limit['type']
            videos = videoss.filter(mime_type=mime_type[type][mime_t])

            while len(availableStream) == 0 and resolution_index > -1:
                availableStream = videos.filter(resolution=resolution[resolution_index])
                if type == 'video' and len(availableStream.filter(fps=60)) != 0:
                    availableStream = availableStream.filter(fps=60)
                    

                resolution_index -= 1

            if len(availableStream) == 0:
                availableStream = self.yt.get_highest_resolution()

        elif res == -1:
            availableStream = self.yt.get_lowest_resolution()

        else:
            availableStream = self.yt.get_highest_resolution()


        availableStream = availableStream[-1]
        if not availableStream.is_progressive and dlaudio:
            self.downloadAudio()
        availableStream.download(output_path='/content/video/', filename=self.id+self.title)
        
        
    def downloadAudio(self):
        audio = self.yt.streams.get_audio_only()
        audio.download(output_path='/content/audio/', filename=self.id+self.title)
        
    def downloadSubtitle(self):
        for i in cpt_lang:
            caption = self.yt.captions.get_by_language_code(i)
            if caption != None:
                caption.download(title=self.id+self.title, srt=False, output_path='/content/subtitle/')


In [ ]:
for output_dir in ['/content/subtitle/', '/content/audio/', '/content/video/']:
    if not os.path.exists(output_dir):
        # os.makedirs 传入一个path路径，生成一个递归的文件夹；如果文件夹存在，就会报错,因此创建文件夹之前，需要使用os.path.exists(path)函数判断文件夹是否存在；
        os.makedirs(output_dir)

In [ ]:
def playlistDownloader(url):
    video_list = Playlist(url).videos
    cnt = 1
    width = ceil(log(len(video_list)+1, 10))


    thread_list = []
    print('analyzing ...')
    test_h = 0
    test_l = 0
    for i in range(3):
        test_num = int(random.random() * (len(video_list)-1))
        if test_num < 0:
            test_num = 0
        test_h += video_list[test_num].streams.get_highest_resolution().filesize/1024/1024
        test_l += video_list[test_num].streams.get_lowest_resolution().filesize/1024/1024
    test_h /= 3
    test_l /= 3
    print("there are %d pieces of video in this playlist"%len(video_list))
    mode = input('the highest resolution takes about %dMB and the lowest resolution takes about %dMB, devide download lists?(y/n):'%(test_h, test_l))
    if mode == 'n':
        usethread = input('enable threads?(y/n)')
        if usethread == 'y':
            for video in video_list:
                while len(thread_list) == maxSubThreadNumber:
                  sleep(5)
                  for i in thread_list:
                    if i.complete:
                      thread_list.remove(i)
                t = videoDownloader(video=video,id=str(cnt).zfill(width))
                thread_list.append(t)
                t.start()
                cnt += 1
                
        elif usethread == 'n':
            for video in video_list:
              t = videoDownloader(video=video,id=str(cnt).zfill(width))
              t.run()
              cnt += 1
    elif mode == 'y':
        start = int(input('start at:')) - 1
        end = int(input('end on:'))
        if start < 0:
            start = 0
        if end > len(video_list):
            end = len(video_list)
        for i in range(start, end):
            try:
                while len(thread_list) == maxSubThreadNumber:
                  sleep(5)
                  for d in thread_list:
                    if d.complete:
                      thread_list.remove(d)
                t = videoDownloader(video_list[i],id=str(i+1).zfill(width))
                thread_list.append(t)
                t.start()
            except Exception as e:
                print(e)
                pass
      
    while thread_list:
      print(str(len(thread_list)) + " task in queue, waiting ...")
      sleep(10)
      for i in thread_list:
        if i.complete:
          thread_list.remove(i)
    print("download complete.")


In [ ]:
limit = {
    "type": "video",
    "mime_type": "mp4"
}

res = 5 # [0,7] higher -> better

mime_type = {
    "video":{"mp4":"video/mp4", "3gpp":"video/3gpp", "webm":"video/webm"},
    "audio":{"mp4":"audio/mp4", "webm":"audio/webm"}
}
# resolution = ["2160p", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"]
resolution = ['144p', '240p', '360p', '480p', '720p', '1080p', '1440p', '2160p']
fps = [60, 30]

cpt_lang = [
    'en',
    'ja',
    'zh-TW',
    'zh-CN'
]

In [ ]:
def single_url():
  url = input("url of a playlist or video: ")
  if 'playlist' in url:
      zip_name = url[34:]
      playlistDownloader(url)
  elif 'watch' in url:
      zip_name = url[32:43]
      vd = videoDownloader(video=YouTube(url))
      vd.start()
      vd.join()

In [14]:
def multi_utl():
    url_list = set()
    url = input("input url of videos or type 'exit' or lick Enter to stop add url")
    while('watch' in url):
        url_list.add(url)
        url = input("input url of videos or type 'exit' or lick Enter to stop add url")
    zip_name = 'video_list'
    print(url_list)

    url_list = list(url_list)

        
    usethread = input('enable threads?(y/n)')
    if usethread == 'y':
        thread_list = []
        for url in url_list:
            vd = videoDownloader(video=YouTube(url))
            while len(thread_list) == maxSubThreadNumber:
                sleep(5)
                for i in thread_list:
                    if i.complete:
                        thread_list.remove(i)
            thread_list.append(vd)
            vd.start()
        while thread_list:
            print(str(len(thread_list)) + " task in queue, waiting ...")
            sleep(10)
            for i in thread_list:
                if i.complete:
                    thread_list.remove(i)
            print("download complete.")
    else:
        for url in url_list:
            vd = videoDownloader(video=YouTube(url))
            vd.run()

In [ ]:
maxSubThreadNumber = 8 # the max numbers of file download at the same time if thread is enabled
zip_name = None
semaLock = None
# count = 0


dlvideo = True
dlaudio = True
dlsubtitle = False

# thread_list = []


if(input("use url list to download video lists(only support urls of videos, not playlist)?(y/n)") == 'n'):
    single_url()
else:
    multi_utl()


In [ ]:
ctn = input('continue?(y/n)')
if ctn != 'y':
    raise Exception('user stop')

In [ ]:
old_name = zip_name
ctn = input('specified zip_name?(input name.../n)')
if ctn and ctn != 'n':
    zip_name = re.sub('[&/:*?"<>| ]','_',ctn)
    print("name: " + zip_name)

In [ ]:
os.system('mkdir ' + zip_name)
os.system('mv ./* ./' + zip_name)

In [ ]:
!pwd

In [ ]:
from IPython.display import FileLink
for dirname, _, filenames in os.walk('/content/%s'%zip_name):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip uninstall aligo -y
!python -m pip install git+https://github.com/lijunjie2232/alg_modify.git

In [ ]:
from aligo import Aligo
# from tqdm.notebook import tqdm
import shutil

In [ ]:
rft = input('refresh_token: ')
ali = Aligo(refresh_token=rft,level=20)

In [ ]:
user = ali.get_user()
print(user.user_name, user.nick_name, user.phone)  # 打印用户信息

In [ ]:
ll = ali.get_file_list()  # 获取网盘根目录文件列表
for file in ll:  # 遍历文件列表
    print(file.file_id, file.name, file.type)  # 打印文件信息

In [ ]:
zip_name

In [ ]:
result = ali.create_folder(zip_name) # make dir on aliyun drive
print(result)

In [ ]:
upload = input("upload?(y/n)")
if upload == 'y':
    is_z = input("excute to zip(this op will lead origin files to be deleted)?(y/n)")
    if is_z == 'y':
      is_z_rc = input("please reconfirm?(y/n)")
      if is_z_rc == 'y':
        os.system("tar -cvjf %s.tar.bz2 ./%s/* --remove-files"%(zip_name, zip_name))
        os.rename(zip_name+'.tar.bz2',zip_name+'.tar.bz2'+'.pdf')
        os.system("mv %s %s" %(zip_name+'.tar.bz2'+'.pdf', zip_name))
    # os.system("rename 's/\.mp4/\.pdf/'  ./%s/*"%zip_name)
    else:
      # os.system('mkdir "/content/%s"' %zip_name)
      # os.system('mv /content/* "/content/%s"' %zip_name)
      for dirname, _, filenames in os.walk('/content/'+zip_name+'/video'):
          for filename in filenames:
            os.rename(os.path.join(dirname, filename),os.path.join(dirname, filename)+'.mp4.pdf')
            os.system('mv "%s" "%s"' % (os.path.join(dirname, filename)+'.mp4.pdf','/content/'+zip_name))

      for dirname, _, filenames in os.walk('/content/'+zip_name+'/subtitle'):
          for filename in filenames:
            os.rename(os.path.join(dirname, filename),os.path.join(dirname, filename)+'.xml.pdf')
            os.system('mv "%s" "%s"' % (os.path.join(dirname, filename)+'.xml.pdf','/content/'+zip_name))

      for dirname, _, filenames in os.walk('/content/'+zip_name+'/audio'):
          for filename in filenames:
            os.rename(os.path.join(dirname, filename),os.path.join(dirname, filename)+'.mp3.pdf')
            os.system('mv "%s" "%s"' % (os.path.join(dirname, filename)+'.mp3.pdf','/content/'+zip_name))
            
      os.system('rm -rf "/content/%s/video"' % zip_name)
      os.system('rm -rf "/content/%s/audio"' % zip_name)
      os.system('rm -rf "/content/%s/subtitle"' % zip_name)
elif upload == 'n':
    os.system("tar -cvjf %s.tar.bz2 ./%s/* --remove-files"%(zip_name, zip_name))
#     FileLink(zip_name + '.tar.bz2')

In [ ]:
upload = input("continue to upload?(y/n)")

if upload == 'y':
    ali.upload_folder('/content/'+zip_name, remove_file_needed=True)

In [ ]:
# FileLink(zip_name + '.tar.bz2')

In [ ]:
# zip_name